In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5

In [2]:
TickDataname='GDF2022_3_27_16_52.csv'
TickGDF=pd.read_csv('CollectedData/'+TickDataname)

TGDF=TickGDF[(TickGDF['TM_GP']>=0)&(TickGDF['TM_GP']<39)]
TGDF=TGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()
new_df=new_df.reset_index()
Train_df=new_df
Train_df.loc[new_df['TM_GP']==0,'시가']=Train_df.loc[new_df['TM_GP']==1,'시가'].values

In [3]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

Train_df[['고가','저가']]=Train_df[['고가','저가']].div(Train_df['시가'],axis=0).values
Train_df[['고가','저가','직전체결가격','시가']]=np.log(Train_df[['고가','저가','직전체결가격','시가']])

In [4]:
qtylist=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량']

Train_df[qtylist]=Train_df[qtylist].div(Train_df['10단계호가합계잔량'],axis=0).values

Train_df['10단계호가합계잔량']=np.log(new_df['10단계호가합계잔량'])

In [5]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
# divdict['시가']=['고가','저가']
loglist=[]
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']


In [6]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [7]:
Train_df[loglist]=np.log(Train_df[loglist])

In [8]:
ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickGDF=pd.read_csv('CollectedData/'+ORDTickDataname)

In [9]:
ORDTickGDF.columns

Index(['Unnamed: 0', 'ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO', 'ASKBID_TP_CD',
       'MODCANCL_TP_CD', 'ORD_QTY', 'ORD_ACPT_TM', 'TM_GP'],
      dtype='object')

In [10]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]

OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

In [11]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [12]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

In [13]:
feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',  '시가', '직전체결가격','NET_ORD_QTY']

In [14]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]
mbrnlist=mbrnlist1+[a for a in mbrnlist2 if a not in mbrnlist1]

In [15]:
Train_df[feat_cols[:-1]]

,ASK_STEP1_BSTORD_RQTY,ASK_STEP2_BSTORD_RQTY,ASK_STEP3_BSTORD_RQTY,ASK_STEP4_BSTORD_RQTY,ASK_STEP5_BSTORD_RQTY,BID_STEP1_BSTORD_RQTY,BID_STEP2_BSTORD_RQTY,BID_STEP3_BSTORD_RQTY,BID_STEP4_BSTORD_RQTY,BID_STEP5_BSTORD_RQTY,매도5단계호가합계잔량,매수5단계호가합계잔량,매도10단계호가합계잔량,매수10단계호가합계잔량,매도총호가잔량,매수총호가잔량,고가,저가,시가,직전체결가격
0,2747.990548,2595.790170,2746.374291,2138.451796,1136.423440,4669.856333,10579.837429,15336.962193,38390.568998,15719.650284,11365.030246,84696.875236,25604.519849,153699.476371,6.013891e+05,289358.451796,2099.877127,2080.018904,2095.0,2087.013233
1,1654.401028,1829.098972,1808.282776,2417.829049,3160.422879,6835.155527,11514.744216,15962.980720,25180.331620,28429.875321,10870.034704,87923.087404,25869.254499,225458.948586,7.635658e+05,350305.340617,2100.000000,2061.272494,2095.0,2071.664524
2,4099.106542,4420.942056,2667.203738,2586.082243,2853.532710,12535.181308,29726.736449,26533.087850,32460.869159,30285.882243,16626.867290,131541.757009,29840.852336,268637.910280,8.648292e+05,387893.278505,2100.000000,2058.943925,2095.0,2066.242991
3,3693.259259,4698.292593,2373.362963,3368.737037,4080.968519,8258.662963,27552.083333,31463.977778,37470.918519,38595.055556,18214.620370,143340.698148,38178.362963,276198.670370,8.870972e+05,421321.096296,2100.000000,2053.500000,2095.0,2062.444444
4,1613.032258,3227.782258,6288.818548,6281.685484,4359.576613,10710.879032,21894.483871,31501.181452,36757.782258,39495.641129,21770.895161,140359.967742,41283.975806,296825.568548,8.943737e+05,453878.294355,2100.000000,2050.000000,2095.0,2064.153226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81895,5861.903654,8832.514286,9976.696346,15666.691694,18417.752159,3561.305648,4203.645183,5115.394684,6469.042525,7511.897010,58755.558140,26861.285050,237869.430565,91729.460465,2.200132e+06,239682.176080,1516.717608,1440.000000,1470.0,1493.661130
81896,9091.382208,16789.253158,17986.322350,25651.784734,24138.342852,8173.281347,11120.823174,5089.531576,8204.235402,4948.547501,93657.085301,37536.419001,332493.707670,70162.490207,2.210884e+06,237266.904265,1520.000000,1440.000000,1470.0,1501.751785
81897,6195.417137,10179.451036,14149.833333,16717.849341,13138.809793,34251.062147,11780.032015,17877.037665,9563.238230,10990.758945,60381.360640,84462.129002,247054.442561,141417.953861,2.204492e+06,273197.397363,1520.000000,1440.000000,1470.0,1492.612994
81898,8091.058568,13754.570499,13148.932755,12163.093275,10865.162690,5033.199566,9727.156182,19084.802603,21331.947939,31589.765727,58022.817787,86766.872017,142376.650759,171490.535792,2.277060e+06,266184.479393,1520.000000,1440.000000,1470.0,1484.490239


In [16]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    print(Train_df['NET_ORD_QTY'].sum())
    ylabel=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0)).values
    

    SaveDirPath='/Data/Min/TransLOB/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    DataSubfix=str(mbr)+'_'+str(brn)+"NoPreProcess"
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix
    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,ylabel)
    with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
        f.write("TrainDataname:"+TrainDataname+'\n')
        f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
        f.write("TickDataname:"+TickDataname+'\n')
        f.write("ORDTickDataname:"+ORDTickDataname+'\n')

        f.write("divdict:"+'\n')
        for col,llist in divdict.items():
            f.write(str(col)+'\t'+str(llist))
        f.write('\n')
        f.write("loglist:"+'\n')
        for col in loglist:
            f.write(str(col)+'\t')
        f.write('\n')
        
        f.write("feat_cols:"+'\n')
        for col in feat_cols:
            f.write(str(col)+'\t')
        f.write('\n')
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)

5 194
-33328534.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_5_194NoPreProcess
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__5_194NoPreProcess
2 155
1011342.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_2_155NoPreProcess
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__2_155NoPreProcess
12 100
-27072832.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_12_100NoPreProcess
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__12_100NoPreProcess
17 29
-27191045.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_17_29NoPreProcess
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__17_29NoPreProcess
42 1
2665621.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_42_1NoPreProcess
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__42_1NoPreProcess
44 1
-17303050.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_44_1NoPreProcess
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__44_1NoPreProcess
50 92
-2678352.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_50_92NoPreProcess
path: /Data/Min/Trans

a

SaveDirPath='/Data/Min/TransLOB/TrainData/'
now=datetime.now()
TrainDataname='Train_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
TrainLabelDataname='Train_Label_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
np.save(SaveDirPath+TrainLabelDataname,increment)
with open(SaveDirPath+'DataInformation.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname)
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')
print("path:",SaveDirPath+TrainDataname)
print("path:",SaveDirPath+TrainLabelDataname)